In [1]:
import torch
from omegaconf import OmegaConf
from src.metoken_model import MeToken_Model
from src.datasets.featurizer import featurize
from inference import get_seq_str, apply_ptm_indices

/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# checkpoint
checkpoint_path = 'pretrained_model/checkpoint.ckpt'
# inference example
pdb_file_path = 'examples/Q16613.pdb'
output_json_path = 'examples/predict.json'
output_path = 'examples/predict.hdf5'
# - predict the PTM type at the certain positions
predict_indices = [31]

In [4]:
# load the data
protein_data = get_seq_str(pdb_file_path)
protein_data = apply_ptm_indices(protein_data, predict_indices)
data = featurize([protein_data])

In [3]:
# load the model
checkpoint = torch.load(checkpoint_path)
params = OmegaConf.load('configs/MeToken.yaml')
model = MeToken_Model(params)
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [5]:
result = model(data)
preds = result['log_probs'].argmax(dim=-1).cpu()

for pos in predict_indices:
    print(f'PTM type at the position {pos} is {preds[pos]}.\n')

PTM type at the position 31 is 24.

